# Capítulo 7 — Intervalos de confianza e inferencia (t-test)

Este notebook acompaña al manual.  
Aquí verás comentarios cortos que explican **sintaxis** y **errores comunes**.


In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

plt.rcParams["figure.figsize"] = (8, 4.5)
plt.rcParams["figure.dpi"] = 120


## Recordatorio de sintaxis

- `np.mean(x)` promedio
- `np.std(x, ddof=1)` desviación estándar muestral (**ojo ddof=1**)
- `se = s/np.sqrt(n)` error estándar del promedio
- `stats.t.ppf(q, df)` cuantil t (para IC/valores críticos)
- `stats.t.cdf(t, df)` prob acumulada (útil para p-value en cola izquierda)
- `stats.ttest_1samp(x, popmean=0)` prueba t automática (p-value bilateral)


## 1) Datos y variación

In [ ]:
TD87 = np.array([10, 1, 6, .45, 1.25, 1.3, 1.06, 3, 8.18, 1.67, .98, 1, .45, 5.03, 8, 9, 18, .28, 7, 3.97])
TD88 = np.array([3, 1, 5, .5, 1.54, 1.5, .8, 2, .67, 1.17, .51, .5, .61, 6.7, 4, 7, 19, .2, 5, 3.83])

# Resta vectorizada (elemento a elemento)
variacion = TD88 - TD87
n = len(variacion)

baseTD = pd.DataFrame({
    "Empresa": np.arange(1, n + 1),
    "TD87": TD87,
    "TD88": TD88,
    "Variación": variacion
})
baseTD.head()


## 2) IC 95% con t (manual)

In [ ]:
# Promedio muestral
promedio = np.mean(variacion)

# Desviación estándar muestral (ERROR común: olvidar ddof=1)
s = np.std(variacion, ddof=1)

# Error estándar del promedio
se = s / np.sqrt(n)

promedio, s, se


In [ ]:
# Cuantil para IC 95% bilateral: 0.975 (NO 0.95)
c = stats.t.ppf(0.975, n - 1)

LI95 = promedio - c * se
LS95 = promedio + c * se
LI95, LS95


## 3) Prueba t (una cola izquierda) manual

In [ ]:
mu0 = 0

# Estadístico t
t_manual = (promedio - mu0) / se

# Valor crítico (cola izquierda 5%)
val_critico = stats.t.ppf(0.05, n - 1)

t_manual, val_critico


In [ ]:
# p-value (cola izquierda): P(T <= t)
p_una_cola = stats.t.cdf(t_manual, n - 1)
p_una_cola


## 4) Prueba t automática (ttest_1samp)

In [ ]:
test = stats.ttest_1samp(variacion, popmean=0)  # bilateral por defecto
t_auto = test.statistic
p_bilateral = test.pvalue

# Para una cola (izquierda), típicamente p/2 si t es negativo
p_una_cola_auto = p_bilateral / 2

t_auto, p_bilateral, p_una_cola_auto


## 5) Ejemplo local: variable y en {-1,0,1}

In [ ]:
audit_like = pd.DataFrame({
    "y": [-1, 0, 0, -1, 1, 0, -1, 0, 0, 1,
          0, 0, -1, 0, 1, 0, 0, -1, 0, 0,
          1, 0, -1, 0, 0, 0, -1, 0, 0, 1]
})

audit_like["y"].value_counts()


In [ ]:
y = audit_like["y"]
n = len(y)

promedio_y = np.mean(y)
s_y = np.std(y, ddof=1)
se_y = s_y / np.sqrt(n)

# Cuantiles normales: 0.975 (95%) y 0.995 (99%)
c95 = stats.norm.ppf(0.975)
c99 = stats.norm.ppf(0.995)

IC95 = (promedio_y - c95 * se_y, promedio_y + c95 * se_y)
IC99 = (promedio_y - c99 * se_y, promedio_y + c99 * se_y)

IC95, IC99


In [ ]:
# Prueba t bilateral sobre y
test_y = stats.ttest_1samp(y, popmean=0)
test_y.statistic, test_y.pvalue
valc_2colas = stats.t.ppf(0.025, n - 1)
print(f"t (automático): {test_y.statistic:.4f}")
print(f"valor crítico (2 colas): {valc_2colas:.4f}")
print(f"valor p (2 colas): {test_y.pvalue:.4f}")



## 6) Datos reales (opcional)

In [ ]:
# En Colab:
# !pip -q install wooldridge
# import wooldridge as woo
# audit = woo.dataWoo("audit")
# y = audit["y"]
# print(stats.ttest_1samp(y, popmean=0))


## Ejercicios propuestos

1) Con `variacion`, calcula el IC 95% usando t.

**Respuesta esperada:** dos números (LI, LS).


In [ ]:
# Escribe tu solución aquí


2) Calcula `t_manual = (promedio-0)/se`.

**Respuesta esperada:** un número.


In [ ]:
# Escribe tu solución aquí


3) Calcula `stats.t.ppf(0.05, n-1)`.

**Respuesta esperada:** un número negativo.


In [ ]:
# Escribe tu solución aquí


4) Calcula el valor p (cola izq) con `stats.t.cdf(t_manual, n-1)`.

**Respuesta esperada:** un número entre 0 y 1.


In [ ]:
# Escribe tu solución aquí


5) Compara `t_manual` con `ttest_1samp(...).statistic`.

**Respuesta esperada:** casi iguales.


In [ ]:
# Escribe tu solución aquí


6) Con `audit_like["y"]`, calcula IC 99% con `stats.norm.ppf(0.995)`.

**Respuesta esperada:** intervalo más ancho que el 95%.


In [ ]:
# Escribe tu solución aquí


## Glosario

- **SE**: error estándar del promedio.
- **ddof=1**: ajuste muestral.
- **ppf**: cuantil (inversa de CDF).
- **valor crítico**: punto de corte.
- **p-value**: probabilidad bajo H0.
